# Data Understanding

Source: [KDD data mining cup 2013 - shopping cart abandonment prediction](https://www.data-mining-cup.com/reviews/dmc-2013/)

## Data Dictionary

- sessionNo: running number of the session
- startHour: hour in which the session has begun
- startWeekday: day of week in which the session has begun (1: Mon, 2: Tue, ..., 7: Sun)
- duration: time in seconds passed since start of the session
- cCount: number of the products clicked on
- cMinPrice: lowest price of a product clicked on
- cMaxPrice: highest price of a product clicked on
- cSumPrice: sum of the prices of all products clicked on
- bCount: number of products put in the shopping basket
- bMinPrice: lowest price of all products put in the shopping basket
- bMaxPrice: highest price of all products put in the shopping basket
- bSumPrice: sum of theprices of all products put in the shopping basket
- bStep: purchase processing step (1,2,3,4,5)
- onlineStatus: indication whether the customer is online
- availability: delivery status
- customerID: customer ID
- maxVal: maximum admissible purchase price for the customer
- customerScore: customer evaluation from the point of view of the shop
- accountLifetime: lifetime of the customer's account in months
- payments: number of payments affected by the customer
- age: age of the customer
- address: form of address of the customer (1: Mr, 2: Mrs, 3: company)
- lastOrder: time in days passed since the last order
- order: outcome of the session (y: purchase, n: non-purchase)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os

sns.set_context('talk')

In [2]:
input_train_path = os.path.join(
    '..',
    'dataset',
    'transact_train.txt'
)

In [3]:
df_train = pd.read_csv(input_train_path, sep="|")
df_train.head()

,sessionNo,startHour,startWeekday,duration,cCount,cMinPrice,cMaxPrice,cSumPrice,bCount,bMinPrice,...,availability,customerNo,maxVal,customerScore,accountLifetime,payments,age,address,lastOrder,order
0,1,6,5,0.000,1,59.99,59.99,59.99,1,59.99,...,?,1,600,70,21,1,43,1,49,y
1,1,6,5,11.940,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
2,1,6,5,39.887,1,59.99,59.99,59.99,1,59.99,...,completely orderable,1,600,70,21,1,43,1,49,y
3,2,6,5,0.000,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y
4,2,6,5,15.633,0,?,?,?,0,?,...,completely orderable,?,?,?,?,?,?,?,?,y


In [4]:
df_train.shape

(429013, 24)

In [5]:
df_train.iloc[0]

sessionNo              1
startHour              6
startWeekday           5
duration             0.0
cCount                 1
cMinPrice          59.99
cMaxPrice          59.99
cSumPrice          59.99
bCount                 1
bMinPrice          59.99
bMaxPrice          59.99
bSumPrice          59.99
bStep                  ?
onlineStatus           ?
availability           ?
customerNo             1
maxVal               600
customerScore         70
accountLifetime       21
payments               1
age                   43
address                1
lastOrder             49
order                  y
Name: 0, dtype: object

In [6]:
df_train.dtypes

sessionNo            int64
startHour            int64
startWeekday         int64
duration           float64
cCount               int64
cMinPrice           object
cMaxPrice           object
cSumPrice           object
bCount               int64
bMinPrice           object
bMaxPrice           object
bSumPrice           object
bStep               object
onlineStatus        object
availability        object
customerNo          object
maxVal              object
customerScore       object
accountLifetime     object
payments            object
age                 object
address             object
lastOrder           object
order               object
dtype: object

In [7]:
df_train.describe()

,sessionNo,startHour,startWeekday,duration,cCount,bCount
count,429013.000000,429013.000000,429013.000000,429013.000000,429013.000000,429013.000000
mean,25274.631293,14.617061,5.924839,1573.901640,24.140317,4.135168
std,14441.366146,4.485914,0.790930,2427.123356,30.398164,4.451778
min,1.000000,0.000000,5.000000,0.000000,0.000000,0.000000
25%,12731.000000,11.000000,5.000000,225.070000,5.000000,1.000000
50%,25470.000000,15.000000,6.000000,738.199000,13.000000,3.000000
75%,37542.000000,18.000000,7.000000,1880.265000,31.000000,5.000000
max,50000.000000,23.000000,7.000000,21580.092000,200.000000,108.000000


No columns have missing values. However, it may contain `?`.

In [8]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 429013 entries, 0 to 429012
Data columns (total 24 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   sessionNo        429013 non-null  int64  
 1   startHour        429013 non-null  int64  
 2   startWeekday     429013 non-null  int64  
 3   duration         429013 non-null  float64
 4   cCount           429013 non-null  int64  
 5   cMinPrice        429013 non-null  object 
 6   cMaxPrice        429013 non-null  object 
 7   cSumPrice        429013 non-null  object 
 8   bCount           429013 non-null  int64  
 9   bMinPrice        429013 non-null  object 
 10  bMaxPrice        429013 non-null  object 
 11  bSumPrice        429013 non-null  object 
 12  bStep            429013 non-null  object 
 13  onlineStatus     429013 non-null  object 
 14  availability     429013 non-null  object 
 15  customerNo       429013 non-null  object 
 16  maxVal           429013 non-null  obje

## Reformat data types + handle missing values

In [9]:
for col in df_train.columns:
    print(f"Preview of {col}")
    print(sorted(df_train[col].unique()[-5:]))
    print("--------------------------------")
    
    ## replace missing value with a numeric value, e.g., -99
    mask = (df_train[col] == '?')
    df_train.loc[mask, col] = -99

Preview of sessionNo
[49996, 49997, 49998, 49999, 50000]
--------------------------------
Preview of startHour
[1, 2, 3, 4, 5]
--------------------------------
Preview of startWeekday
[5, 6, 7]
--------------------------------
Preview of duration
[675.114, 715.341, 2961.909, 4700.383, 5988.882]
--------------------------------
Preview of cCount
[134, 164, 183, 189, 199]
--------------------------------
Preview of cMinPrice
['14.96', '159.2', '175.0', '29.75', '5.4']
--------------------------------
Preview of cMaxPrice
['175.0', '201.99', '225.0', '35.9', '809.9']
--------------------------------
Preview of cSumPrice
['2974.6', '4297.08', '4315.03', '5253.28', '667.88']
--------------------------------
Preview of bCount
[104, 105, 106, 107, 108]
--------------------------------
Preview of bMinPrice
['175.0', '2999.99', '39.16', '5.4', '52.4']
--------------------------------
Preview of bMaxPrice
['175.0', '34.5', '35.9', '5.4', '52.4']
--------------------------------
Preview of bSumPr

In [10]:
# then, convert to numeric
numeric_column_list = [
    'startHour', 'startWeekday', 'duration', 'cCount',
   'cMinPrice', 'cMaxPrice', 'cSumPrice', 'bCount', 'bMinPrice',
   'bMaxPrice', 'bSumPrice', 'bStep',
   'customerNo', 'maxVal', 'customerScore', 'accountLifetime', 'payments',
   'age', 'address', 'lastOrder'
]

for col in numeric_column_list:
    df_train[col] = df_train[col].astype(float)

In [14]:
for col in ['onlineStatus','order']:
    condition_list = [
        (df_train[col] == 'y'),
        (df_train[col] == 'n'),
    ]
    choice_list = [
        1,
        0
    ]
    df_train[col] = np.select(condition_list, choice_list, default=df_train[col])

In [15]:
df_train['order'].value_counts(True)

1    0.67604
0    0.32396
Name: order, dtype: float64

In [16]:
df_train['bStep'].value_counts()

-99.0    191333
 1.0      90058
 2.0      60682
 4.0      41142
 3.0      30062
 5.0      15736
Name: bStep, dtype: int64

In [17]:
df_train.iloc[0]

sessionNo              1
startHour            6.0
startWeekday         5.0
duration             0.0
cCount               1.0
cMinPrice          59.99
cMaxPrice          59.99
cSumPrice          59.99
bCount               1.0
bMinPrice          59.99
bMaxPrice          59.99
bSumPrice          59.99
bStep              -99.0
onlineStatus         -99
availability         -99
customerNo           1.0
maxVal             600.0
customerScore       70.0
accountLifetime     21.0
payments             1.0
age                 43.0
address              1.0
lastOrder           49.0
order                  1
Name: 0, dtype: object

In [19]:
SAVE_DATA = False

if SAVE_DATA:
    output_train_path = os.path.join(
        '..',
        'dataset',
        'preprocessed',
        'transact_train.csv'
    )
    df_train.to_csv(output_train_path, index=False)

## Exploratory Analysis

In [21]:
df_train.columns

Index(['sessionNo', 'startHour', 'startWeekday', 'duration', 'cCount',
       'cMinPrice', 'cMaxPrice', 'cSumPrice', 'bCount', 'bMinPrice',
       'bMaxPrice', 'bSumPrice', 'bStep', 'onlineStatus', 'availability',
       'customerNo', 'maxVal', 'customerScore', 'accountLifetime', 'payments',
       'age', 'address', 'lastOrder', 'order'],
      dtype='object')